In [1]:
import geopandas as gpd
from glob import glob
import pandas as pd
DATADIR = '/data/uscuni-ulce/cadastre_buildings_raw/'

In [2]:
regions = [
    
# "buildings_austria*",
# "buildings_slovakia*",
# "buildings_poland*",
# "buildings_czechia*",
]

In [3]:
eubucco_regions = [
    # '/data/uscuni-ulce/eubucco_raw/v0_1-AUT*',
    # '/data/uscuni-ulce/eubucco_raw/v0_1-SVK*',
    # '/data/uscuni-ulce/eubucco_raw/v0_1-POL*',
    # '/data/uscuni-ulce/eubucco_raw/v0_1-CZE*',
]


In [11]:
for region, eubucco_region in zip(regions, eubucco_regions):
    
    files = glob(DATADIR + region)
    frames = []
    for f in files:
        gdf = gpd.read_parquet(f)
        if gdf.shape[0]:
            frames.append(gdf.to_crs(epsg=3035))
    gdf = pd.concat(frames, ignore_index=True)


    files = glob(eubucco_region)
    frames = []
    for f in files:
        gdf = gpd.read_file(f)
        if gdf.shape[0]:
            frames.append(gdf)
    eubucco_gdf = pd.concat(frames, ignore_index=True)

    print(gdf.shape, eubucco_gdf.shape)

/tmp/ipykernel_108951/1962546803.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gdf = pd.concat(frames, ignore_index=True)


(1746587, 7) (43644887, 7)


In [5]:
f

'/data/uscuni-ulce/cadastre_buildings_raw/buildings_germany_th_2900000.pq'

In [50]:
# (4427904, 30) (4135733, 7)
# (3563020, 31) (3488125, 7)
# (15387645, 14) (14404767, 7)
# (4274434, 36) (4044659, 7)

In [ ]:
# bad - hessen(broken counts), mv(continue from where it broke), sachsen-anhalt ( wrooong), th (wroong)
## sh(schesig hollstein is sg)

In [230]:
ger_eubucco = '/data/uscuni-ulce/eubucco_raw/v0_1-DEU_*'
ger_regions = ['bavaria', 'brandeburg', 'bw', 'hessen', 'mv',
       'nrw', 'rlp', 'saarland', 'sachsen', 'sachsen-anhalt', 'sg', 
       'th', 'ni', 'bremen', 'hamburg', 'berlin']
len(ger_regions)

16

In [266]:
region = ger_regions[4]
region

'mv'

In [272]:
files = glob(f'{DATADIR}buildings_germany_{region}*')

In [273]:
sorted(files)[:10]

['/data/uscuni-ulce/cadastre_buildings_raw/buildings_germany_mv_0.pq',
 '/data/uscuni-ulce/cadastre_buildings_raw/buildings_germany_mv_10000.pq',
 '/data/uscuni-ulce/cadastre_buildings_raw/buildings_germany_mv_100000.pq',
 '/data/uscuni-ulce/cadastre_buildings_raw/buildings_germany_mv_1000000.pq',
 '/data/uscuni-ulce/cadastre_buildings_raw/buildings_germany_mv_1010000.pq',
 '/data/uscuni-ulce/cadastre_buildings_raw/buildings_germany_mv_1020000.pq',
 '/data/uscuni-ulce/cadastre_buildings_raw/buildings_germany_mv_1030000.pq',
 '/data/uscuni-ulce/cadastre_buildings_raw/buildings_germany_mv_1040000.pq',
 '/data/uscuni-ulce/cadastre_buildings_raw/buildings_germany_mv_1050000.pq',
 '/data/uscuni-ulce/cadastre_buildings_raw/buildings_germany_mv_1060000.pq']

In [274]:
len(files)

152

In [275]:
frames = []
for f in files:
    gdf = gpd.read_parquet(f)
    if gdf.shape[0]:
        frames.append(gdf.to_crs(epsg=3035))
gdf = pd.concat(frames, ignore_index=True)
if 'gml_id' in gdf.columns:
    gdf = gdf[~gdf['gml_id'].duplicated()]

In [276]:
gdf.shape

(1511715, 13)

In [277]:
import requests
import re
def get_size(url, params):
    params = params.copy()
    params['resultType'] = 'hits'
    response = requests.get(url, params=params)
    if response.status_code == 200:
        size = re.findall(r'numberMatched="[0-9]+"', str(response.content))
        size = int(size[0].split('=')[-1][1:-1])
        return size

    return 'unknown'

In [278]:
params = {
        "SERVICE": "WFS",
        "REQUEST": "GetFeature",
        "TYPENAMES": "ave:GebaeudeBauwerk",
        "SRSNAME": "EPSG:5650",
        "VERSION": '2.0.0',
    }
url = 'https://www.geodaten-mv.de/dienste/alkis_wfs_einfach'

size = get_size(url, params)
print(size)

1511715


In [6]:
gdf = gpd.read_parquet(files[0])

In [18]:
new_gdf = gdf.set_crs(epsg=25832).to_crs(epsg=3035)

In [21]:
# new_gdf.iloc[100000:101000].explore()

In [37]:
frames = []
for f in files:
    frames.append(gpd.read_parquet(f).to_crs(epsg=3035))
gdf = pd.concat(frames, ignore_index=True)

KeyboardInterrupt: 

In [ ]:
gdf

In [10]:
gpd.read_parquet(f)

,gml_id,identifier,beginLifespanVersion,localId,namespace,percentage,referenceGeometry,horizontalGeometryEstimatedAccuracy,numberOfFloorsAboveGround,end,dateOfRenovation|DateOfEvent|end,geometry
0,Building_DEHHALKA3wT000D2,https://registry.gdi-de.org/id/de.hh/e33c193f-...,2018-05-24T08:48:56Z,Building_DEHHALKA3wT000D2,https://registry.gdi-de.org/id/de.hh/e33c193f-...,None,True,None,NaN,None,None,"MULTIPOLYGON (((581432.726 5919754.877, 581422..."
1,Building_DEHHALKA3wT000D0,https://registry.gdi-de.org/id/de.hh/e33c193f-...,2018-05-24T08:48:56Z,Building_DEHHALKA3wT000D0,https://registry.gdi-de.org/id/de.hh/e33c193f-...,None,True,None,NaN,None,None,"MULTIPOLYGON (((581308.145 5919081.784, 581298..."
2,Building_DEHHALKA3xh000Id,https://registry.gdi-de.org/id/de.hh/e33c193f-...,2018-05-31T10:07:45Z,Building_DEHHALKA3xh000Id,https://registry.gdi-de.org/id/de.hh/e33c193f-...,None,True,None,NaN,None,None,"MULTIPOLYGON (((581781.689 5918892.378, 581780..."
3,Building_DEHHALKA3D1000Ih,https://registry.gdi-de.org/id/de.hh/e33c193f-...,2017-08-15T09:52:04Z,Building_DEHHALKA3D1000Ih,https://registry.gdi-de.org/id/de.hh/e33c193f-...,None,True,None,NaN,None,None,"MULTIPOLYGON (((580328.395 5918984.083, 580319..."
4,Building_DEHHALKA3xh000Is,https://registry.gdi-de.org/id/de.hh/e33c193f-...,2018-05-31T10:07:45Z,Building_DEHHALKA3xh000Is,https://registry.gdi-de.org/id/de.hh/e33c193f-...,None,True,None,NaN,None,None,"MULTIPOLYGON (((581781.426 5918910.251, 581770..."
...,...,...,...,...,...,...,...,...,...,...,...,...
454566,Building_DEHHALKAn0007tva,https://registry.gdi-de.org/id/de.hh/e33c193f-...,2012-04-02T07:54:53Z,Building_DEHHALKAn0007tva,https://registry.gdi-de.org/id/de.hh/e33c193f-...,None,True,None,1.0,None,None,"MULTIPOLYGON (((462544.986 5977856.458, 462550..."
454567,Building_DEHHALKA4YR00007,https://registry.gdi-de.org/id/de.hh/e33c193f-...,2019-01-11T05:21:25Z,Building_DEHHALKA4YR00007,https://registry.gdi-de.org/id/de.hh/e33c193f-...,None,True,None,1.0,2017-12-31T23:00:00Z,None,"MULTIPOLYGON (((463420.799 5979454.284, 463415..."
454568,Building_DEHHALKA4YR00003,https://registry.gdi-de.org/id/de.hh/e33c193f-...,2019-07-03T13:37:37Z,Building_DEHHALKA4YR00003,https://registry.gdi-de.org/id/de.hh/e33c193f-...,None,True,None,1.0,2017-12-31T23:00:00Z,None,"MULTIPOLYGON (((463423.607 5979446.542, 463420..."
454569,Building_DEHHALKAn0007tvb,https://registry.gdi-de.org/id/de.hh/e33c193f-...,2012-04-02T07:54:53Z,Building_DEHHALKAn0007tvb,https://registry.gdi-de.org/id/de.hh/e33c193f-...,None,True,None,1.0,None,None,"MULTIPOLYGON (((463251.829 5979353.284, 463242..."
